# 1) Euclidean Distance 

In [230]:
import pandas as pd
import geopandas as gpd


In [231]:
# display supermarkets' coordinates
supermarkets_df = pd.read_csv('../../data/curated/preprocessed_supermarkets.csv') 
supermarkets_df.head()

,Supermarket,Latitude,Longitude,Suburb,Postcode,Address
0,IGA,-37.277362,144.733861,No Suburb,No Postcode,NaN
1,Coles,-37.791434,145.171842,No Suburb,No Postcode,55 Tunstall Square
2,Foodworks,-37.788987,144.976090,No Suburb,No Postcode,799 Nicholson Street
3,Coles,-37.854786,145.182650,No Suburb,No Postcode,495-511 Burwood Highway
4,Coles,-37.868257,145.240404,No Suburb,No Postcode,NaN


In [232]:
# display properties' coordinates
properties_df = pd.read_csv('../../notebooks/Aarav/property_details_with_longlat.csv')
properties_df.head()

,title,description,street_address,suburb,postcode,price,bedrooms,bathrooms,parking,primary_property_type,structured_features,video_count,photo_count,date_listed,days_listed,floor_plans_count,virtual_tour,nearby_schools,latitude,longitude
0,"60 Little Windrock Lane, Craigieburn VIC 3064 ...","View this 2 bedroom, 1 bathroom rental house a...","60 Little Windrock Lane, Craigieburn VIC 3064",Craigieburn,3064,$450 Per Week,2.0,1.0,1.0,House,"[{'category': 'Indoor', 'name': 'Built in ward...",0.0,21.0,2024-08-22T16:07:26.000,14.0,0.0,False,"[{'address': 'Craigieburn, VIC 3064', 'distanc...",-37.588897,144.915516
1,"53 Were Street, Brighton VIC 3186 - House For ...","View this $1,500/week 4 bedroom, 2 bathroom re...","53 Were Street, Brighton VIC 3186",Brighton,3186,"$1,490.00",4.0,2.0,2.0,House,[],0.0,6.0,2024-06-02T18:11:41.000,95.0,2.0,True,"[{'address': 'Brighton, VIC 3186', 'distance':...",-37.925640,144.999904
2,"43 Tackle Drive, Point Cook VIC 3030 - Townhou...","View this 3 bedroom, 2 bathroom rental townhou...","43 Tackle Drive, Point Cook VIC 3030",Point Cook,3030,$550 per Week,3.0,2.0,2.0,Townhouse/Villa,"[{'category': 'Outdoor', 'name': 'Secure Parki...",0.0,17.0,2024-09-03T12:01:18.000,2.0,0.0,True,"[{'address': 'Point Cook, VIC 3030', 'distance...",-37.906257,144.720254
3,"3 Rostrevor Parade, Mont Albert VIC 3127 - Hou...","View this 5 bedroom, 2 bathroom rental house a...","3 Rostrevor Parade, Mont Albert VIC 3127",Mont Albert,3127,$800 weekly,5.0,2.0,2.0,House,[],0.0,8.0,2024-07-01T12:53:48.000,66.0,0.0,False,"[{'address': 'Mont Albert, VIC 3127', 'distanc...",-37.812918,145.106110
4,"48 Roberts Street, Frankston VIC 3199 - Studio...","View this 9 bedroom, 3 bathroom rental studio ...","48 Roberts Street, Frankston VIC 3199",Frankston,3199,$299 per week,9.0,3.0,4.0,Apartment,"[{'category': 'Indoor', 'name': 'Furnished', '...",0.0,20.0,2024-07-02T11:24:10.000,65.0,1.0,False,"[{'address': 'Frankston, VIC 3199', 'distance'...",-38.154913,145.140409


In [233]:
# convert supermarkets and properties into GeoDataFrames Point geometry 
supermarkets_gdf = gpd.GeoDataFrame(supermarkets_df, 
                                    geometry=gpd.points_from_xy(supermarkets_df['Longitude'], supermarkets_df['Latitude']), 
                                    crs="EPSG:4326")
properties_gdf = gpd.GeoDataFrame(properties_df, 
                                  geometry=gpd.points_from_xy(properties_df['longitude'], properties_df['latitude']), 
                                  crs="EPSG:4326")

In [234]:
def closest_supermarket_distance(property_point, supermarkets_gdf):
    '''
    Calculates distance of each houses to the closest supermarket in Victoria.
    
    Parameters:
    - property_point : shapely.geometry.Point = location of each house property
    - supermarkets_gdf : geopandas.GeoDataFrame = supermarkets location in Victoria
    
    Returns:
    - tuple = (minimum Euclidean distance, supermarket geometry)
    '''
    # Calculate the distance between the property and all supermarkets
    distances = supermarkets_gdf.geometry.distance(property_point)
    
    # coordinates of the closest supermarket
    closest_supermarket_geometry = supermarkets_gdf.loc[distances.idxmin()].geometry
    
    # Return the minimum distance and supermarket coordinates
    return distances.min(), closest_supermarket_geometry

In [235]:
# convert to Metres distance units 
supermarkets_gdf.to_crs(epsg=3857, inplace=True)
properties_gdf.to_crs(epsg=3857, inplace=True)

In [236]:
# Calculate supermarket's distance (in meters) to each property
properties_gdf['distance_to_closest_supermarket'] = properties_gdf.geometry.apply(
    lambda x: closest_supermarket_distance(x, supermarkets_gdf)[0]
)
properties_gdf['closest_supermarket_geometry'] = properties_gdf.geometry.apply(
    lambda x: closest_supermarket_distance(x, supermarkets_gdf)[1]
)

In [237]:
properties_gdf.head()

,title,description,street_address,suburb,postcode,price,bedrooms,bathrooms,parking,primary_property_type,...,date_listed,days_listed,floor_plans_count,virtual_tour,nearby_schools,latitude,longitude,geometry,distance_to_closest_supermarket,closest_supermarket_geometry
0,"60 Little Windrock Lane, Craigieburn VIC 3064 ...","View this 2 bedroom, 1 bathroom rental house a...","60 Little Windrock Lane, Craigieburn VIC 3064",Craigieburn,3064,$450 Per Week,2.0,1.0,1.0,House,...,2024-08-22T16:07:26.000,14.0,0.0,False,"[{'address': 'Craigieburn, VIC 3064', 'distanc...",-37.588897,144.915516,POINT (16131921.46 -4521512.425),491.097565,POINT (16131864.843 -4522000.248)
1,"53 Were Street, Brighton VIC 3186 - House For ...","View this $1,500/week 4 bedroom, 2 bathroom re...","53 Were Street, Brighton VIC 3186",Brighton,3186,"$1,490.00",4.0,2.0,2.0,House,...,2024-06-02T18:11:41.000,95.0,2.0,True,"[{'address': 'Brighton, VIC 3186', 'distance':...",-37.925640,144.999904,POINT (16141315.445 -4568926.558),1910.765178,POINT (16140520.713 -4567188.909)
2,"43 Tackle Drive, Point Cook VIC 3030 - Townhou...","View this 3 bedroom, 2 bathroom rental townhou...","43 Tackle Drive, Point Cook VIC 3030",Point Cook,3030,$550 per Week,3.0,2.0,2.0,Townhouse/Villa,...,2024-09-03T12:01:18.000,2.0,0.0,True,"[{'address': 'Point Cook, VIC 3030', 'distance...",-37.906257,144.720254,POINT (16110184.994 -4566191.508),1381.567610,POINT (16110905.854 -4565012.911)
3,"3 Rostrevor Parade, Mont Albert VIC 3127 - Hou...","View this 5 bedroom, 2 bathroom rental house a...","3 Rostrevor Parade, Mont Albert VIC 3127",Mont Albert,3127,$800 weekly,5.0,2.0,2.0,House,...,2024-07-01T12:53:48.000,66.0,0.0,False,"[{'address': 'Mont Albert, VIC 3127', 'distanc...",-37.812918,145.106110,POINT (16153138.221 -4553030.95),1069.539360,POINT (16153045.358 -4554096.451)
4,"48 Roberts Street, Frankston VIC 3199 - Studio...","View this 9 bedroom, 3 bathroom rental studio ...","48 Roberts Street, Frankston VIC 3199",Frankston,3199,$299 per week,9.0,3.0,4.0,Apartment,...,2024-07-02T11:24:10.000,65.0,1.0,False,"[{'address': 'Frankston, VIC 3199', 'distance'...",-38.154913,145.140409,POINT (16156956.429 -4601332.969),612.948151,POINT (16157002.487 -4601944.184)


Manual conversion of Supermarket's geometry to Degrees unit (epsg=4326)

In [238]:
# check that the supermarket is in GeoSeries and in EPSG:3857
supermarket_geo_series = gpd.GeoSeries(properties_gdf['closest_supermarket_geometry'])
supermarket_geo_series.set_crs(epsg=3857, inplace=True)

# Update the properties_gdf with the converted EPSG:4326 geometries
properties_gdf['closest_supermarket_geometry'] = supermarket_geo_series.to_crs(epsg=4326)

In [239]:
# verify that it is in degrees
properties_gdf['closest_supermarket_geometry']

0        POINT (144.91501 -37.59237)
1        POINT (144.99276 -37.91333)
2         POINT (144.72673 -37.8979)
3        POINT (145.10528 -37.82048)
4        POINT (145.14082 -38.15923)
                    ...             
11031    POINT (144.70327 -37.88159)
11032    POINT (144.44116 -37.65947)
11033    POINT (144.57082 -37.72217)
11034    POINT (145.04809 -38.24535)
11035    POINT (145.08441 -37.99716)
Name: closest_supermarket_geometry, Length: 11036, dtype: geometry

In [240]:
# convert euclidean distance to EPSG:4326 to satisfy API condition later
supermarkets_gdf.to_crs(epsg=4326, inplace=True)
properties_gdf.to_crs(epsg=4326, inplace=True)

In [241]:
# verify that the location is now in degrees
properties_gdf.head()

,title,description,street_address,suburb,postcode,price,bedrooms,bathrooms,parking,primary_property_type,...,date_listed,days_listed,floor_plans_count,virtual_tour,nearby_schools,latitude,longitude,geometry,distance_to_closest_supermarket,closest_supermarket_geometry
0,"60 Little Windrock Lane, Craigieburn VIC 3064 ...","View this 2 bedroom, 1 bathroom rental house a...","60 Little Windrock Lane, Craigieburn VIC 3064",Craigieburn,3064,$450 Per Week,2.0,1.0,1.0,House,...,2024-08-22T16:07:26.000,14.0,0.0,False,"[{'address': 'Craigieburn, VIC 3064', 'distanc...",-37.588897,144.915516,POINT (144.91552 -37.5889),491.097565,POINT (144.91501 -37.59237)
1,"53 Were Street, Brighton VIC 3186 - House For ...","View this $1,500/week 4 bedroom, 2 bathroom re...","53 Were Street, Brighton VIC 3186",Brighton,3186,"$1,490.00",4.0,2.0,2.0,House,...,2024-06-02T18:11:41.000,95.0,2.0,True,"[{'address': 'Brighton, VIC 3186', 'distance':...",-37.925640,144.999904,POINT (144.9999 -37.92564),1910.765178,POINT (144.99276 -37.91333)
2,"43 Tackle Drive, Point Cook VIC 3030 - Townhou...","View this 3 bedroom, 2 bathroom rental townhou...","43 Tackle Drive, Point Cook VIC 3030",Point Cook,3030,$550 per Week,3.0,2.0,2.0,Townhouse/Villa,...,2024-09-03T12:01:18.000,2.0,0.0,True,"[{'address': 'Point Cook, VIC 3030', 'distance...",-37.906257,144.720254,POINT (144.72025 -37.90626),1381.567610,POINT (144.72673 -37.8979)
3,"3 Rostrevor Parade, Mont Albert VIC 3127 - Hou...","View this 5 bedroom, 2 bathroom rental house a...","3 Rostrevor Parade, Mont Albert VIC 3127",Mont Albert,3127,$800 weekly,5.0,2.0,2.0,House,...,2024-07-01T12:53:48.000,66.0,0.0,False,"[{'address': 'Mont Albert, VIC 3127', 'distanc...",-37.812918,145.106110,POINT (145.10611 -37.81292),1069.539360,POINT (145.10528 -37.82048)
4,"48 Roberts Street, Frankston VIC 3199 - Studio...","View this 9 bedroom, 3 bathroom rental studio ...","48 Roberts Street, Frankston VIC 3199",Frankston,3199,$299 per week,9.0,3.0,4.0,Apartment,...,2024-07-02T11:24:10.000,65.0,1.0,False,"[{'address': 'Frankston, VIC 3199', 'distance'...",-38.154913,145.140409,POINT (145.14041 -38.15491),612.948151,POINT (145.14082 -38.15923)


# 2) OpenRouteService API

In [242]:
import requests
import time

In [243]:
# Michele's openroute API key
api_key = '5b3ce3597851110001cf62482137f179f8c64044a4f02291f1598019'

In [244]:
def calculate_driving_distance(lat1, lon1, lat2, lon2, api_key):
    url = 'https://api.openrouteservice.org/v2/directions/driving-car'
    params = {
        'api_key': api_key,
        # properties first as 'start' location then supermarkets for 'end' 
        'start': f'{lon1},{lat1}',  
        'end': f'{lon2},{lat2}'     
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()

        if 'features' in data and len(data['features']) > 0:
            # get driving distance (in meters) from API  
            distance = data['features'][0]['properties']['segments'][0]['distance']
            return distance 
        else:
            print(f"Error in response: {data}")
            time.sleep(10)  
            return None
    # error handling
    except Exception as e:
        print(f"Error calculating distance: {e}")
        return None


In [245]:
# 339 total properties with Euclidean distance greater than 10km
len(properties_gdf[properties_gdf['distance_to_closest_supermarket'] > 10000])

339

We only have 2000 API requests per day, so we need to batch the driving distance to those with extremely large Euclidean distance only (greater than 10km). 

We assume that properties with Euclidean distance less than 10km has closely similar driving distance. Thus, we only need to process 339 properties' driving distance.

*Code below takes ~17.5 mins*

In [247]:
# set threshold for Euclidean distance (in meters)
euclidean_threshold = 10000  

# set driving distance equal to small euclidean distance 
properties_gdf.loc[properties_gdf['distance_to_closest_supermarket'] <= euclidean_threshold, 
                   'driving_distance_to_closest_supermarket'] = properties_gdf['distance_to_closest_supermarket']


# find driving distance for properties with large distance only
large_distance_properties = properties_gdf[properties_gdf['distance_to_closest_supermarket'] > euclidean_threshold]

# find properties with large Euclidean distance only
for idx, row in large_distance_properties.iterrows():
    # retrieve coordinates for each supermarket and property
    property_lon = row['geometry'].x
    property_lat = row['geometry'].y
    supermarket_lon = row['closest_supermarket_geometry'].x
    supermarket_lat = row['closest_supermarket_geometry'].y

    # driving distance using the API
    driving_distance = calculate_driving_distance(property_lat, property_lon, supermarket_lat, supermarket_lon, api_key)
    
    # store results for large driving distance
    properties_gdf.at[idx, 'driving_distance_to_closest_supermarket'] = driving_distance
    
    # Add delay to avoid exceeding the API request limit
    time.sleep(1.5)

Error in response: {'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 147.0559844 -36.0480243.'}, 'info': {'engine': {'build_date': '2024-09-13T08:01:06Z', 'version': '8.1.3'}, 'timestamp': 1726822006573}}
Error in response: {'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 145.8182660 -37.5222245.'}, 'info': {'engine': {'build_date': '2024-09-13T08:01:06Z', 'version': '8.1.3'}, 'timestamp': 1726822457667}}
Error in response: {'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 146.2300940 -37.1664244.'}, 'info': {'engine': {'build_date': '2024-09-13T08:01:06Z', 'version': '8.1.3'}, 'timestamp': 1726822745409}}


In [248]:
# check that euclidean distance is equal to driving distance
properties_gdf[properties_gdf['distance_to_closest_supermarket'] <= 10000].head()

,title,description,street_address,suburb,postcode,price,bedrooms,bathrooms,parking,primary_property_type,...,days_listed,floor_plans_count,virtual_tour,nearby_schools,latitude,longitude,geometry,distance_to_closest_supermarket,closest_supermarket_geometry,driving_distance_to_closest_supermarket
0,"60 Little Windrock Lane, Craigieburn VIC 3064 ...","View this 2 bedroom, 1 bathroom rental house a...","60 Little Windrock Lane, Craigieburn VIC 3064",Craigieburn,3064,$450 Per Week,2.0,1.0,1.0,House,...,14.0,0.0,False,"[{'address': 'Craigieburn, VIC 3064', 'distanc...",-37.588897,144.915516,POINT (144.91552 -37.5889),491.097565,POINT (144.91501 -37.59237),491.097565
1,"53 Were Street, Brighton VIC 3186 - House For ...","View this $1,500/week 4 bedroom, 2 bathroom re...","53 Were Street, Brighton VIC 3186",Brighton,3186,"$1,490.00",4.0,2.0,2.0,House,...,95.0,2.0,True,"[{'address': 'Brighton, VIC 3186', 'distance':...",-37.925640,144.999904,POINT (144.9999 -37.92564),1910.765178,POINT (144.99276 -37.91333),1910.765178
2,"43 Tackle Drive, Point Cook VIC 3030 - Townhou...","View this 3 bedroom, 2 bathroom rental townhou...","43 Tackle Drive, Point Cook VIC 3030",Point Cook,3030,$550 per Week,3.0,2.0,2.0,Townhouse/Villa,...,2.0,0.0,True,"[{'address': 'Point Cook, VIC 3030', 'distance...",-37.906257,144.720254,POINT (144.72025 -37.90626),1381.567610,POINT (144.72673 -37.8979),1381.567610
3,"3 Rostrevor Parade, Mont Albert VIC 3127 - Hou...","View this 5 bedroom, 2 bathroom rental house a...","3 Rostrevor Parade, Mont Albert VIC 3127",Mont Albert,3127,$800 weekly,5.0,2.0,2.0,House,...,66.0,0.0,False,"[{'address': 'Mont Albert, VIC 3127', 'distanc...",-37.812918,145.106110,POINT (145.10611 -37.81292),1069.539360,POINT (145.10528 -37.82048),1069.539360
4,"48 Roberts Street, Frankston VIC 3199 - Studio...","View this 9 bedroom, 3 bathroom rental studio ...","48 Roberts Street, Frankston VIC 3199",Frankston,3199,$299 per week,9.0,3.0,4.0,Apartment,...,65.0,1.0,False,"[{'address': 'Frankston, VIC 3199', 'distance'...",-38.154913,145.140409,POINT (145.14041 -38.15491),612.948151,POINT (145.14082 -38.15923),612.948151


In [249]:
# check that euclidean distance is NOT equal to driving distance
properties_gdf[properties_gdf['distance_to_closest_supermarket'] > 10000].head()

,title,description,street_address,suburb,postcode,price,bedrooms,bathrooms,parking,primary_property_type,...,days_listed,floor_plans_count,virtual_tour,nearby_schools,latitude,longitude,geometry,distance_to_closest_supermarket,closest_supermarket_geometry,driving_distance_to_closest_supermarket
19,"7/36 Clyde Street, Jindabyne NSW 2627 - Apartm...","View this 2 bedroom, 1 bathroom rental apartme...","36 Clyde Street, Jindabyne NSW 2627",Jindabyne,2627,$375/week - Summer Lease,2.0,1.0,1.0,Apartment,...,8.0,0.0,False,"[{'address': 'Jindabyne, NSW 2627', 'distance'...",-36.415132,148.613347,POINT (148.61335 -36.41513),84445.807518,POINT (147.90558 -36.19514),237593.7
33,"1/9 Compton Street, Port Macdonnell SA 5291 - ...","View this 2 bedroom, 1 bathroom rental apartme...","9 Compton Street, Port Macdonnell SA 5291",Port Macdonnell,5291,$210 pw,2.0,1.0,1.0,Apartment,...,38.0,0.0,False,"[{'address': 'Allendale East, SA 5291', 'dista...",-38.055683,140.688524,POINT (140.68852 -38.05568),105419.526852,POINT (141.63071 -38.13075),108160.6
46,"88 Potoroo Avenue, Thurgoona NSW 2640 - House ...","View this 4 bedroom, 2 bathroom rental house a...","88 Potoroo Avenue, Thurgoona NSW 2640",Thurgoona,2640,$650 per week,4.0,2.0,2.0,House,...,3.0,0.0,False,"[{'address': 'Thurgoona, NSW 2640', 'distance'...",-36.063162,147.008277,POINT (147.00828 -36.06316),15444.120912,POINT (146.88884 -36.1202),17265.2
50,"156 Hotham Circuit, Thurgoona NSW 2640 - House...","View this 2 bedroom, 1 bathroom rental house a...","156 Hotham Circuit, Thurgoona NSW 2640",Thurgoona,2640,$430 per week,2.0,1.0,1.0,House,...,3.0,0.0,False,"[{'address': 'Thurgoona, NSW 2640', 'distance'...",-36.045754,146.990595,POINT (146.99059 -36.04575),15279.764389,POINT (146.88884 -36.1202),14154.9
53,"615 Cape Otway Road, Moriac VIC 3240 - House F...","View this 3 bedroom, 1 bathroom rental house a...","615 Cape Otway Road, Moriac VIC 3240",Moriac,3240,$670 per week,3.0,1.0,9.0,House,...,6.0,1.0,False,"[{'address': 'Moriac, VIC 3240', 'distance': 6...",-38.243159,144.168297,POINT (144.1683 -38.24316),18583.230320,POINT (144.32903 -38.20775),18303.5


### Fill in missing driving distance

In [250]:
# Check for rows where 'driving_distance_to_closest_supermarket' is NaN
empty_distance_rows = properties_gdf[properties_gdf['driving_distance_to_closest_supermarket'].isna()]

# Print the number of rows where driving distance is empty
print(f"Total properties with missing driving distance: {len(empty_distance_rows)}")
empty_distance_rows

Total properties with missing driving distance: 3


,title,description,street_address,suburb,postcode,price,bedrooms,bathrooms,parking,primary_property_type,...,days_listed,floor_plans_count,virtual_tour,nearby_schools,latitude,longitude,geometry,distance_to_closest_supermarket,closest_supermarket_geometry,driving_distance_to_closest_supermarket
2919,"Wirlinga NSW 2640 - 4 beds house for Rent, $69...","View this 4 bedroom, 3 bathroom rental house i...",Wirlinga NSW 2640,Wirlinga,2640,$690 per week,4.0,3.0,6.0,House,...,28.0,0.0,False,"[{'address': 'Thurgoona, NSW 2640', 'distance'...",-36.048024,147.055984,POINT (147.05598 -36.04802),21096.177328,POINT (146.88884 -36.1202),NaN
7202,"Marysville VIC 3779 - 3 beds house for Rent, $...","View this $2,500/week 3 bedroom, 2 bathroom re...",Marysville VIC 3779,Marysville,3779,$2500.00,3.0,2.0,1.0,House,...,17.0,1.0,False,"[{'address': 'Marysville, VIC 3779', 'distance...",-37.522225,145.818266,POINT (145.81827 -37.52222),35156.685606,POINT (145.69684 -37.75309),NaN
10516,"217 Desmonds Road, Boorolite VIC 3723 - House ...","View this 2 bedroom, 1 bathroom rental house a...","217 Desmonds Road, Boorolite VIC 3723",Boorolite,3723,Under Application,2.0,1.0,2.0,House,...,90.0,0.0,False,"[{'address': 'Merrijig, VIC 3723', 'distance':...",-37.166424,146.230094,POINT (146.23009 -37.16642),22304.524633,POINT (146.09016 -37.05206),NaN


Properties with empty driving distances has Euclidean distance greater than 15km, so the properties are assumed to , be in remote areas where the API cannot find a nearby supermarket. Thus we will fill missing values with the Euclidean distance as an approximation for these.

In [251]:
# Fill missing driving distance with Euclidean distance 
properties_gdf['driving_distance_to_closest_supermarket'].fillna(properties_gdf['distance_to_closest_supermarket'], inplace=True)

# Verify that there's no more missing values
print(properties_gdf['driving_distance_to_closest_supermarket'].isna().sum())


0


In [252]:
# save into `data/curated` directory 
properties_gdf.to_csv("../../data/curated/property_details_with_distance_to_closest_supermarkets.csv", index=False)